# Analiza vpliva centralizacije na občine v Sloveniji

Cilj naloge je ugotoviti katere občine bi pridobile največ z decentralizacijo delovnih mest v Sloveniji. V nalogi bom analiziral migracije med Slovenskimi občinami in se naslajal na podatke o zaslužkih ljudi v danih občinah, njihovih migracijah ter cene nepremičnin. S temi podatki bom ugotavljal v katerih občinah bi bilo mogoče najbolj izboljšat zaslužek in standard življenja. 

## Uvoz podatkov

Na začetku naloge bom iz zbranih podatkov izračunal kazalnike, ki vplivajo na posamezno občino. S tem namenom naložimo vse podatke, ki jih bomo pri analizi uporabili.

In [1]:
import pandas as pd

# Nastavitve prikaza
pd.options.display.max_rows = 20

%matplotlib inline

# Podatki
import os.path

PODATKI = "./podatki"

nepremicnine = pd.read_csv(os.path.join(PODATKI, "nepremicnine.csv"), index_col="id")
podjetja = pd.read_csv(os.path.join(PODATKI, "podjetja.csv"), index_col="id")
razdalje = pd.read_csv(os.path.join(PODATKI, "razdalje.csv"), index_col="zacetek")
migracije = pd.read_csv(os.path.join(PODATKI, "sistat/migracije.csv"))
migracije_v_obcino = pd.read_csv(os.path.join(PODATKI, "migracije_v_obcino.csv"), index_col=[0])
migracije_iz_obcine = pd.read_csv(os.path.join(PODATKI, "migracije_iz_obcine.csv"), index_col=[0])
obcine = pd.read_csv(os.path.join(PODATKI, "obcine.csv"), index_col="ime")

# Pomožne vrednosti
vektor_obcin = pd.Series(obcine.index, index=obcine.index)

Poglejmo najprej s kakšnimi podatki bomo delali. Naložili smo podatke o nepremičninah, ki vsebujejo podatke o vrsti nepremičnine, ceni, vrsti posredovanja, številu sob in podobno...; prav tako sem zbral podatke o razdaljah med posameznimi občinami in migracijah iz posamezne občine v druge občine in v posamezno občino iz drugih občin, ki nam bodo pomagali ocenit koliko povprečno posameznik izgubi časa in denarja zaradi vožnje na delo.

Poleg teh podatkov, imamo še informacije o večjih Slovenskih podjetjih, kar nam bo pomagalo ocenit katera podjetja delajo največjo izgubo s trenutno ureditvijo.

Vse podatke pa bomo združevali po občinah, zato imamo tudi statistične podatke o posamezni občini. Podatke o nepremičninah, podjetjih in migracijah sem s pomočjo Levenshteinovega algoritma popravil, da se občine ujemajo z občinami iz statističnega urada RS.

## Obdelava podatkov o nepremičninah

Najprej izračunajmo povprečno vrednosti nepremičnin v posamezni občini. Pri tem se osredotočimo na hiše in stanovanja, ki se prodajajo in si zabeležimo še koliko nepremičnin prištejemo k povprečju. To nam bo pomagalo ocenit kako dobra je naša ocena.

Cene nepremičnin nam bodo pomagale ocenit, koliko migrantje v posamezni občini prišparajo z nakupom nepremičnine zunaj občine v kateri delajo. Za izračun teh podatkov bom uporabil podatke o povprečni ceni nepremičnin v dani občini in verjetnosti, da občan dane občine  dela v poljubni drugi občini. Ker želim izračunati relativno ceno nepremičnine glede na posamezno občino bom gledal razliko v ceni nepremičnine med občino v kateri smo in občino v kateri migrant dela.

In [2]:
# Vsem nepremičninam izračunamo ceno kvadratnega metra.
nepremicnine["cena_na_m2"] = nepremicnine.cena / nepremicnine.velikost

# Zajem
nepremicnine_po_obcinah = nepremicnine[
    nepremicnine.vrsta.isin(["Hiša", "Stanovanje"]) & 
    (nepremicnine.posredovanje == "prodaja")
].groupby("obcina")
    
# Obdelaj in priključi podatke k občinam.
obcine["cena_na_m2"] = nepremicnine_po_obcinah["cena_na_m2"].mean()
obcine["stevilo_nepremicnin"] = nepremicnine_po_obcinah.size()

# Ocena razlike v ceni.
def relativna_cena_na_m2(obcina):
    """Vrne relativno razliko cene nepremičnin v občini glede na druge občine."""
    
    relativne_cene = (obcine.loc[obcina].cena_na_m2 - obcine.cena_na_m2) * migracije_iz_obcine[obcina]
    return relativne_cene.sum()

obcine["relativna_cena_na_m2"] = vektor_obcin.apply(relativna_cena_na_m2)

## Obdelava podatkov o migracijah

Izračunajmo sedaj kako Slovenci migriramo med občinami. Zanima nas, kakšen delež migrantov iz posamezne regije gre povprečno v dano drugo regijo. Povedano drugače, zanima nas kakšna je verjetnost, da oseba, ki živi v Ljubljani in migrira, migrira v Maribor, Koper...

Za lažjo obdelavo podatkov sem verjetnosti o premikih izračunal v posebnem programu in jih uvozim kot tabeli o migracijah iz obcine in v obcino.

### Izračun stroškov migracije

Na podlagi verjetnosti, da občani dane občine migrirajo v poljubno drugo občino, bom izračunal koliko časa in kakšno pot, v povprečju, prebivalec poljubne občine prevozi. Podatke o povprečnih vrednostih časa in razdalje migracije iz dane občine bom nato priključil tabeli občin.

In [3]:
# Izračunamo koliko časa in kilometrov v povprečju posameznik iz dane občine porabi.
def pot(obcina):
    migracije = migracije_iz_obcine[obcina]
    poti = razdalje[razdalje.index == obcina].set_index("konec")

    df = pd.DataFrame({ 
        "razdalja": poti.razdalja, 
        "cas": poti.cas, 
        "migracije": migracije
    }, index=migracije.index)

    df["rel_razdalja"] = df.razdalja * df.migracije
    df["rel_cas"] = df.cas * df.migracije
    
    return pd.Series(
        [
            df["rel_razdalja"].sum(), 
            df["rel_cas"].sum()
        ], 
        index=["razdalja", "cas"]
    )


# Izračunaj na vseh občinah.
poti = pd.DataFrame(vektor_obcin.apply(pot), columns=["razdalja", "cas"])

# Dodaj k občinam
obcine["razdalja_migracije"] = poti.razdalja
obcine["cas_migracije"] = poti.cas

Na podlagi podatkov o migracijah lahko ocenimo koliko denarja posameznik v občini nameni za migracije. 

To bom naredil tako, da bom na podlagi povprečne dolžine poti do delovnega mesta, izračunal koliko bencina se porabi za pot ter iz cene naftnih derivatov izračunal oceno potnih stroškov. Na podlagi povprečne neto plače v občini bom izračunal še urno postavko delavca ter upošteval povprečni čas vožnje na delovno mesto za oceno vrednosti časa.

Povprečno porabo avtomobila sem pobral iz [te strani](https://www.iea.org/reports/fuel-consumption-of-cars-and-vans) in znaša malo več kot 7L/100km, kar bom zaokrožil na 7L/100km. Povprečne cene goriva v zadnjih štririh letih sem pobral iz [strani](https://www.energetika-portal.si/statistika/statisticna-podrocja/cene-naftnih-derivatov/). Na podlagi danih cen, sem se za oceno odločil vzeti 1.1€/L, čeprav je, po vsej verjetnosti, cena goriva malce višja.

Izračunal bom še znesek nadomestila, ki ga migrantje prejmejo od svojih podjetji kot obračun prevoza na delo. Povrnjen znesek, kot sem najšel v [viru](https://mladipodjetnik.si/novice-in-dogodki/novice/obracun-prevoz-na-delo-ter-kilometrine-2), znaša 0.18€/km.

Stroške bom izračunal za obdobje enega mesca za vsakega posameznika, saj so tudi delavci v večini primerov plačani za eno-mesečno obdobje.

In [4]:
# Stroški prevoza

poraba_goriva = 7 / 100
cena_goriva = 1.1 / 1
visina_povracila_stroskov = 0.18 / 1

delovni_dnevi_v_mescu = 4 * 5
st_potovanj = delovni_dnevi_v_mescu * 2

# Izračuni vrednosti

povprecna_razdalja_migracij = obcine.razdalja_migracije / 1000
obcine["stroski_goriva"] = povprecna_razdalja_migracij * poraba_goriva * cena_goriva * st_potovanj

# Čas potovanja
delovne_ure_v_mescu = delovni_dnevi_v_mescu * 8
obcine["povprecna_urna_postavka"] = obcine.placa_neto / delovne_ure_v_mescu

povprecen_cas_migracij = obcine.cas_migracije / 3600
obcine["stroski_prevoza"] = povprecen_cas_migracij * st_potovanj * obcine.povprecna_urna_postavka

# Povračilo potnih stroškov na prebivalca.
obcine["povracilo_stroskov_migracije_delavca"] =  povprecna_razdalja_migracij * st_potovanj * visina_povracila_stroskov

# Združimo vse stroške.
obcine["stroski_migracije_delavca"] = obcine.stroski_prevoza + obcine.stroski_goriva
obcine["razlika_v_stroskih_delavca"] = obcine.povracilo_stroskov_migracije_delavca - obcine.stroski_migracije_delavca

## Obdelava podatkov o podjetjih

Ocenili bi radi kakšno izgubo delajo podjetja zaradi plačevanja potnih stroškov svojih delavcev. To bomo naredili tako, da bomo za vsako podjetje ocenili koliko stroškov plača na enega zaposlenega glede na to iz katere občine v povprečju prihaja.

Predpostavil bom, da imajo podjejta vsa delovna mesta v občini registracije in ocenil katera podjetja zaradi prevoza svojih delavcev plačajo največ potnih stroškov. Za boljšo oceno, bom nastavil tudi najmanjšo oddaljenost občine, saj predpostavljam, da podjetja ne plačajo vseh potnih stroškov blizu živečih.

In [5]:
# Nastavitve parametrov

ODDALJENOST = 50_000

# Stroški prevoza delavca za podjetje v posamezni občini.
def potni_stroski(obcina):
    """
    Vrne informacijo o potnih stroških posameznika, ki pride delat
    v dano občino.
    Za boljši približek, občine, ki so blizu naši občini filtriramo.
    """
    
    poti = razdalje[razdalje.index == obcina].set_index("konec")
    index_daljnih_obcin = poti[poti.razdalja > ODDALJENOST].index
    
    daljne_obcine = obcine[obcine.index.isin(index_daljnih_obcin)]
    stroski = migracije_v_obcino[obcina] * daljne_obcine.povracilo_stroskov_migracije_delavca
    
    return stroski.sum()

# Stroški podjetja za prevoz delavcev.
podjetja["potni_stroski"] = podjetja.obcina.apply(potni_stroski) * podjetja.st_zaposlenih

Da bom podatke lahko primerjal med občinami, bom izračunal koliko denarja prebivalci dane občine dobijo v obliki potnih stroškov skupaj. To pomeni, koliko denarja podjetja iz cele Slovenije v povprečju za potne stroške dajo v posamezno občino.

In [6]:
# Izguba zaradi potnih stroškov.
obcine["stroski_migracij"] = podjetja.groupby("obcina").potni_stroski.sum()


# Izračun potnih stroškov, ki se zlivajo v posamezno občino.
def skupno_povracilo(obcina):
    """
    Vrne informacijo o povračilu skupnih potnih stroškov, ki jih delavci iz
    dane občine prejmejo vsak mesec.
    """
    povracila = migracije_v_obcino[obcina] * obcine["stroski_migracij"]
    
    return povracila.sum()
    
obcine["priliv_stroskov_migracij"] = vektor_obcin.apply(skupno_povracilo)

## Zaključek

Na podlagi ocene potnih stroškov v dani občini lahko ocenim v katero občino je najboljše postavit coworking-spacee. V najboljšo občino se izteka največ denarja, saj ga lahko podjetje s coworking-spaci bolj efektivno uporabi in izboljša življenjski standard tamkajšnjih občanov pri tem pa zasluži.

Skozi raziskavo lahko opazimo tudi, da delavci v večini primerov dobijo povrnjene vse stroške svojih poti. Predvidevamo lahko, da ljudje iz posameznih občin ne bodo pripravljeni plačati veliko denarja za delo blizu svojega doma - razliko lahko prodajalec storitve naredi z izpostavljanjem dodatnega časa z družino.

In [8]:
parametri = [
    "cena_na_m2",
    "stevilo_nepremicnin",
    "priliv_stroskov_migracij"
]

obcine.sort_values("priliv_stroskov_migracij", ascending=False)[parametri].head(5)

,cena_na_m2,stevilo_nepremicnin,priliv_stroskov_migracij
ime,,,
Osilnica,333.333333,1.0,3.785600e+06
Kranjska Gora,2608.584132,55.0,3.712999e+06
Kostel,2092.648621,13.0,3.539966e+06
Bohinj,2667.491885,65.0,3.534076e+06
Luče,1364.039346,146.0,3.514588e+06


Namenimo še nekaj besed ovrednotenju rezultatov. Vidimo lahko, da se je v skupnem povračilu pojavila napaka saj je skupna vsota vseh prilivov občin v Sloveniji petkrat večja kot vsota vseh izdatkov. Večina vrednosti je bila ocenjena za namen okvirnega vpogleda v situacijo v Sloveniji.

Očitno je mogoče narediti še velio izboljšav v raziskavi. Sigurno pa je to dober začetek. Zanimivo bi bilo izračunati še kolikšen bi moral bit dodatek k plači glede na odaljenost od delovnega mesta za obdobje 10 let in katere občine delajo največjo izgubo z dnevnimi migracijami.